# NOMAD Samples Dashboard

This dashboard provides a modern interface for managing and analyzing HySprint samples in NOMAD. Key features:

1. **Authentication**: Secure connection to NOMAD Oasis instances
2. **Sample Management**: View and manage HySprint sample data
3. **Author Attribution**: Track and override sample attributions
4. **Analytics**: Visualize sample statistics and trends

## Setup and Dependencies

In [22]:
# Import required libraries
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Text, Password, Label, Dropdown
from IPython.display import display, clear_output, FileLink
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from datetime import datetime, timedelta
import asyncio
from IPython.lib.backgroundjobs import BackgroundJobManager
jobs = BackgroundJobManager()

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Import NOMAD modules
%run 'nomad_auth.ipynb'
%run 'nomad_data_retrieval.ipynb'
%run 'nomad_attribution.ipynb'
%run 'nomad_visualization.ipynb'
from nomad_data import load_attributions, save_attributions

## Authentication Tab

In [23]:
def create_auth_tab():
    """Create the authentication tab using nomad_auth functionality"""
    auth_ui = widgets.VBox([
        widgets.HTML("<h2>General Settings</h2>"),
        oasis_dropdown,
        selected_url_display,
        auth_box  # From nomad_auth.ipynb
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='0 0 20px 0'))
    
    # Create wrapper for authentication state
    auth_state = {
        'is_authenticated': lambda: api_client is not None,
        'token': lambda: current_token,
        'user_info': lambda: current_user_info,
        'client': lambda: api_client,
        'oasis': lambda: oasis_dropdown.value if oasis_dropdown.value else None,
        'oasis_url': lambda: oasis_options.get(oasis_dropdown.value, None) if oasis_dropdown.value else None
    }
    
    return auth_ui, auth_state

## Data Retrieval Tab

In [24]:
def initialize_data_tab(auth_state):
    """Create the data retrieval tab using the modular nomad_data_retrieval functionality"""
    # Use the modular data retrieval component from nomad_data_retrieval.ipynb
    return create_data_tab(auth_state)

## Attribution Management Tab

In [25]:
def initialize_attribution_tab(data_state):
    """Create the attribution tab using the modular nomad_attribution functionality"""
    # Use the modular attribution component from nomad_attribution.ipynb
    return create_attribution_tab(data_state)

## Visualization Tab

In [26]:
def initialize_visualization_tab(data_state):
    """Create the visualization tab using the modular nomad_visualization functionality"""
    return create_visualization_tab(data_state)

## Cache Management Tab

In [27]:
def format_timestamp(ts_str):
    """Format timestamp string to readable format"""
    if not ts_str:
        return 'N/A'
    try:
        dt = datetime.fromisoformat(ts_str)
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except:
        return ts_str

def create_cache_tab():
    """Create the cache management tab"""
    from nomad_data import get_cache_stats, clear_cache, CACHE_CONFIG

    # Create widgets for cache management
    stats_output = widgets.Output()
    status_output = widgets.Output()

    # Create buttons for each cache type
    clear_buttons = {
        cache_type: widgets.Button(
            description=f'Clear {cache_type}',
            button_style='danger',
            layout={'width': '150px'}
        )
        for cache_type in CACHE_CONFIG.keys()
    }

    # Clear all button
    clear_all_button = widgets.Button(
        description='Clear All Cache',
        button_style='danger',
        icon='trash',
        layout={'width': '150px'}
    )

    # Refresh button
    refresh_button = widgets.Button(
        description='Refresh Stats',
        button_style='info',
        icon='sync',
        layout={'width': '150px'}
    )

    def update_stats():
        """Update the cache statistics display"""
        with stats_output:
            clear_output()
            stats = get_cache_stats()
            
            # Create a formatted table of stats
            print("Cache Statistics:")
            print("-" * 80)
            print(f"{'Cache Type':<15} {'Items':<8} {'Size (KB)':<12} {'Oldest':<25} {'Newest':<25}")
            print("-" * 80)
            
            for cache_type, cache_stats in stats.items():
                print(f"{cache_type:<15} {cache_stats['count']:<8} {cache_stats['size_kb']:<12.2f} ",
                      f"{format_timestamp(cache_stats['oldest']):<25} {format_timestamp(cache_stats['newest']):<25}")

    def on_clear_click(cache_type):
        def handler(b):
            with status_output:
                clear_output()
                print(f"Clearing {cache_type} cache...")
                clear_cache(cache_type)
                print(f"✓ {cache_type} cache cleared")
                update_stats()
        return handler

    def on_clear_all_click(b):
        with status_output:
            clear_output()
            print("Clearing all cache...")
            clear_cache()
            print("✓ All cache cleared")
            update_stats()

    def on_refresh_click(b):
        with status_output:
            clear_output()
            print("Refreshing cache statistics...")
            update_stats()
            print("✓ Statistics updated")

    # Set up button handlers
    for cache_type, button in clear_buttons.items():
        button.on_click(on_clear_click(cache_type))
    clear_all_button.on_click(on_clear_all_click)
    refresh_button.on_click(on_refresh_click)

    # Create the HTML for cache expiration times
    cache_expiry_html = "<ul>"
    for k, v in CACHE_CONFIG.items():
        cache_expiry_html += f"<li><b>{k}:</b> {v.get('expire_hours')} hours</li>"
    cache_expiry_html += "</ul>"

    # Create the cache management UI
    cache_ui = widgets.VBox([
        widgets.HTML("<h2>Cache Management</h2>"),
        widgets.HTML(f"<p>Cache expiration times:{cache_expiry_html}</p>"),
        widgets.HBox([refresh_button, clear_all_button]),
        widgets.HBox(list(clear_buttons.values())),
        status_output,
        stats_output
    ])

    # Initialize stats display
    update_stats()

    return cache_ui

## Main Dashboard

In [ ]:
def create_dashboard():
    """Create and display the main dashboard with dynamic tab widths."""

    # Create authentication tab - this will now be a top-level section
    auth_ui, auth_state = create_auth_tab()

    # Create data tab using the modular component
    data_tab, data_state = initialize_data_tab(auth_state)

    # Create attribution tab
    attribution_tab = initialize_attribution_tab(data_state)

    # Create visualization tab
    viz_tab = create_visualization_tab(data_state)

    # Create cache management tab
    cache_tab = create_cache_tab()

    # Create tab widget - note auth_tab is removed from here
    tab_widget = widgets.Tab([data_tab, attribution_tab, viz_tab, cache_tab])

    # Set tab titles - note auth tab removed
    tab_widget.set_title(0, "Data Retrieval")
    tab_widget.set_title(1, "Attribution Management")
    tab_widget.set_title(2, "Visualizations")
    tab_widget.set_title(3, "Cache Management")

    # --- REVISED CSS for Dynamic Tab Width ---
    # Inject CSS to style the tab headers.
    tab_styling = widgets.HTML("""
    <style>
    /* Try targeting common classes directly */
    .lm-TabBar-tab, .p-TabBar-tab, .jp-TabBar-tab {
        min-width: 100px !important;   /* Keep a minimum width */
        width: auto !important;        /* Allow expansion based on content */
        max-width: none !important;    /* Ensure no max-width is limiting it */
        flex: 0 0 auto !important;     /* Equivalent to flex-grow:0, flex-shrink:0, flex-basis:auto */
        padding-left: 15px !important; /* Add horizontal padding */
        padding-right: 15px !important;/* Add horizontal padding */
        white-space: nowrap !important; /* Prevent text wrapping */
        overflow: visible !important;  /* Ensure content isn't clipped */
        border: 1px solid #ccc !important; /* Add border for visibility if needed */
        margin-right: 2px !important; /* Add spacing between tabs */
    }

    /* Target the label inside specifically */
    .lm-TabBar-tab-label, .p-TabBar-tab-label, .jp-TabBar-tab-label {
        white-space: nowrap !important; /* Prevent text wrapping */
        overflow: visible !important;   /* Ensure label text isn't clipped */
        display: inline-block !important; /* Helps with sizing */
    }
    </style>
    """)

    # Create app layout with width constraints
    app_layout = widgets.Layout(
        max_width="1400px",  # Set maximum width
        margin="0 auto",     # Center the layout
        padding='15px'       # Add some padding
    )

    # Create dashboard container with header, auth section, and tabs
    dashboard = widgets.VBox(
        [
            widgets.HTML("""
        <div style="background-color: #4CAF50; color: white; padding: 10px; text-align: center; border-radius: 5px;">
            <h1>NOMAD Samples Dashboard</h1>
            <p>Manage and analyze HySprint samples in NOMAD</p>
        </div>
        """),
            auth_ui,  # Add the auth UI section at the top level
            tab_styling,  # Add the CSS styling widget
            tab_widget,  # Add the tab widget
        ],
        layout=app_layout  # Apply the constrained layout
    )

    return dashboard


# --- Run the Dashboard ---
# Create and display the dashboard
dashboard = create_dashboard()
display(dashboard)


Loaded 24 attribution overrides
